# 마스크 언어 모델 Fine-Tuning
- [강좌링크](https://wikidocs.net/166833)

**domain adaptation**: 분야 특화 데이터에 대해 사전 학습된 언어 모델을 fine-tuning하는 프로세스

## MLM pretrained model 선택하기

BERT 계열 중 다운스트림 성능 손실이 거의 없거나 전혀 없이 훨씬 빠르게 학습될 수 있는 DistilBERT 사용.
지식 증류(knownledge distillation)라는 특별한 기술은 큰 teacher 모델이 훨씬 작은 student 모델의 학습을 안내한다.

In [16]:
from transformers import AutoModelForMaskedLM

model_checkpoint = "distilbert-base-uncased"
model = AutoModelForMaskedLM.from_pretrained(model_checkpoint)

distilbert_num_parameters = model.num_parameters() / 1_000_000
print(f"DistilBERT number of parameters: {round(distilbert_num_parameters)}M")
print(f"BERT number of parameters: 110M")

DistilBERT number of parameters: 67M
BERT number of parameters: 110M


In [17]:
text = "This is a great [MASK]."

from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

import torch

inputs = tokenizer(text, return_tensors="pt")
token_logits = model(**inputs).logits

# MASK의 위치를 찾고 해당 logits 추출
mask_token_index = torch.where(torch.tensor(inputs["input_ids"] == tokenizer.mask_token_id))[1]
mask_token_logits = token_logits[0, mask_token_index, :]

top_5_tokens = torch.topk(mask_token_logits, 5, dim = 1).indices[0].tolist()

for token in top_5_tokens:
    print(f"{text.replace(tokenizer.mask_token, tokenizer.decode([token]))}")

This is a great deal.
This is a great success.
This is a great adventure.
This is a great idea.
This is a great feat.


/tmp/ipykernel_4293/1862350943.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  mask_token_index = torch.where(torch.tensor(inputs["input_ids"] == tokenizer.mask_token_id))[1]


## 데이터셋

domain adaptation 과정을 보여주기 위해 감정 분석 모델을 벤치마킹하는데 자주 사용되는 영화 리뷰 모음인 유명한 **Large Movie Review Dataset, (IMDb)**를 사용한다. 이 데이터를 사용해 DistilBERT를 fine-tuning함으로써 언어모델이 사전 학습된 wikipedia의 사실적인 데이터에서 영화 리뷰의 보다 주관적인 요소에 맞게 어휘를 조정할 것으로 기대한다.

In [18]:
from datasets import load_dataset

imdb_dataset = load_dataset("imdb")
imdb_dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

In [19]:
sample = imdb_dataset["train"].shuffle(seed = 42).select(range(3))

for row in sample:
    print(f">>> Review: {row['text']}")
    print(f">>> Label: {row['label']}")

>>> Review: There is no relation at all between Fortier and Profiler but the fact that both are police series about violent crimes. Profiler looks crispy, Fortier looks classic. Profiler plots are quite simple. Fortier's plot are far more complicated... Fortier looks more like Prime Suspect, if we have to spot similarities... The main character is weak and weirdo, but have "clairvoyance". People like to compare, to judge, to evaluate. How about just enjoying? Funny thing too, people writing Fortier looks American but, on the other hand, arguing they prefer American series (!!!). Maybe it's the language, or the spirit, but I think this series is more English than American. By the way, the actors are really good and funny. The acting is not superficial at all...
>>> Label: 1
>>> Review: This movie is a great. The plot is very true to the book which is a classic written by Mark Twain. The movie starts of with a scene where Hank sings a song with a bunch of kids called "when you stub your 

## 데이터 전처리

auto-regressive modeling과 masked language modeling의 공통 preprocess는 모든 예제를 통합한 다음 전체 말뭉치를 동일한 크기의 청크로 분할하는 것이다. 이는 개별 예제가 너무 길면 절단되어 유용한 정보가 손실될 수 있기 때문이며 따라서 토크나이저에 `truncation=True` 옵션을 **설정하지 않는다.**

In [20]:
def tokenize_function(examples):
    result = tokenizer(examples["text"])
    if tokenizer.is_fast:
        # i번째의 word_ids를 리스트로 반환
        result["word_ids"] = [result.word_ids(i) for i in range(len(result["input_ids"]))]
    return result

# 빠른 멀티스레딩을 작동시키기 위해 batched=True
# 원본 text와 필요없는 label 칼럼은 제거
tokenized_datasets = imdb_dataset.map(
    tokenize_function, batched = True, remove_columns = ["text", "label"]
)
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids'],
        num_rows: 50000
    })
})

In [21]:
tokenizer.model_max_length

512

In [22]:
# 토큰화된 학습 집합에서 랜덤 리뷰를 가져와 리뷰당 토큰 수 확인
tokenized_samples = tokenized_datasets["train"][:3]

for idx, sample in enumerate(tokenized_samples["input_ids"]):
    print(f">>> Review {idx} length: {len(sample)}")

>>> Review 0 length: 363
>>> Review 1 length: 304
>>> Review 2 length: 133


In [23]:
concatenated_exaples = {
    k: sum(tokenized_samples[k], []) for k in tokenized_samples.keys()
}
total_length = len(concatenated_exaples["input_ids"])
print(f">>> Concatenated reviews length: {total_length}")

>>> Concatenated reviews length: 800


전체 길이가 확인되었으니 연결된 리뷰를 `block_size`로 지정된 크기의 청크로 분할한다.

In [24]:
# 학습용이니 chunk size를 작게 지정함
chunk_size = 128

chunks = {
    k: [t[i : i + chunk_size] for i in range(0, total_length, chunk_size)]
    for k, t in concatenated_exaples.items()
}

for chunk in chunks["input_ids"]:
    print(f">>> Chunk length: {len(chunk)}")

>>> Chunk length: 128
>>> Chunk length: 128
>>> Chunk length: 128
>>> Chunk length: 128
>>> Chunk length: 128
>>> Chunk length: 128
>>> Chunk length: 32


In [25]:
r"""
위에서처럼 마지막 청크는 일반적으로 최대 청크 크기보다 작으며 이를 처리하기 위한 방법은 아래와 같다:
- truncation
- padding

여기서는 truncation을 적용하여 위의 예시 로직들을 단일 함수로 구성한다.
"""
def group_texts(examples):
    # 모든 텍스트 결합
    concatenated_exaples = {
        k : sum(examples[k], []) for k in examples.keys()
    }
    
    # 결합된 텍스트들에 대한 길이를 구한다.
    total_length = len(concatenated_exaples[list(examples.keys())[0]])
    
    # chunk_size보다 작은 경우 마지막 청크 삭제
    total_length = (total_length // chunk_size) * chunk_size
    result = {
        k : [t[i : i + chunk_size] for i in range(0, total_length, chunk_size)]
        for k, t in concatenated_exaples.items()
    }
    
    # 새로운 칼럼 레이블 생성
    r"""
    마스킹 된 언어 모델링의 목표는 입력 배치에서 무작위로 마스킹된 토큰을 예측하는 것이고 labels 열이 정답 역할을 하기 때문이다.
    """
    result["labels"] = result["input_ids"].copy()
    return result

In [26]:
lm_datasets = tokenized_datasets.map(group_texts, batched = True, num_proc = 8)
lm_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 61290
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 59899
    })
    unsupervised: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 122952
    })
})

In [27]:
print(tokenizer.decode(lm_datasets["train"][1]["input_ids"]))
print(tokenizer.decode(lm_datasets["train"][1]["labels"]))

as the vietnam war and race issues in the united states. in between asking politicians and ordinary denizens of stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men. < br / > < br / > what kills me about i am curious - yellow is that 40 years ago, this was considered pornographic. really, the sex and nudity scenes are few and far between, even then it's not shot like some cheaply made porno. while my countrymen mind find it shocking, in reality sex and nudity are a major staple in swedish cinema. even ingmar bergman,
as the vietnam war and race issues in the united states. in between asking politicians and ordinary denizens of stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men. < br / > < br / > what kills me about i am curious - yellow is that 40 years ago, this was considered pornographic. really, the sex and nudity scenes are few and far between, even then it's not shot li

## Trainer API를 이용한 DistilBERT Fine-Tuning

3장의 시퀀스 분류 모델을 fine-tuning하는 것과 거의 동일하지만 텍스트의 각 배치에서 일부 토큰을 무작위로 마스킹할 수 있는 특수 데이터 수집기가 필요하다는 점이 다르다. 이 작업은 `DataCollatorForLanguageModeling`클래스가 수행하며 토큰화와 마스킹 토큰의 비율인 `mlm_probability` 인수를 전달하기만 하면 된다.

BERT에서와 같이 논문에서 일반적으로 사용하는 수치인 15%로 지정하고 수행해본다.

In [28]:
from transformers import DataCollatorForLanguageModeling

# 무작위 masking 실행
data_collator = DataCollatorForLanguageModeling(tokenizer = tokenizer, mlm_probability = 0.15)

In [29]:
# test
samples = [lm_datasets["train"][i] for i in range(2)]

test_samples = [
		{k: s[k] for k in s.keys() if k != "word_ids"}
		for s in samples
]
# for sample in samples:
#     _ = sample.pop("word_ids")

for chunk in data_collator(test_samples)["input_ids"]:
    print(f">>> {tokenizer.decode(chunk)}")

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


>>> [CLS] i [MASK] i am curious - yellow from my video store [MASK] of [MASK] the controversy that surrounded it when it was first released in 1967. i also heard that at first it was seized by u. [MASK]. customs if it ever tried to [MASK] this country, therefore being a fan of films considered collection controversial " i really had to see this for myself. < br / > < br / > the plot is centered around a [MASK] swedish drama [MASK] named recording who wants to learn everything she caniji life. in particular she wants to focus her attentions to making some sort of documentary on what the average swede thought about certain political issues such
>>> as the vietnam war and race [MASK] in the united states [MASK] in between asking politicians and [MASK] denizens of stockholm about their opinions on [MASK], she [MASK] sex with her drama teacher, classmates, and married men. < br / [MASK] < br / > what kills polling about i [MASK] curious - yellow is that 40 years ago, this was [MASK] pornogr

무작위 masking의 한 가지 부작용은 학습 및 테스트 집합에 대해 동일한 data collator를 사용하기 때문에 `Trainer`를 사용할 때 평가 메트릭이 결정적이지 않다는 것이다. 이는 아래에 &#129303;Accelerate로 Fine-Tuning할 때 사용자 지정 평가 루프의 유연성을 사용해 임의성을 고정하는 방법을 알아본다.

마스킹 된 언어 모델링을 위해 모델 학습 시 사용할 수 있는 한 가지 기법은 개별 토큰뿐만 아니라 전체 단어를 함께 마스킹하는 것이다. 이를 전체 단어 마스킹(whole word masking)이라고 한다. 이를 사용하려면 data collator를 직접 구현해야 한다. data collator는 샘플 목록을 가져와 일괄 처리로 변환하는 함수에 불과하다. 이제 이를 구현한다.

이전에 계산된 word ids를 사용해 단어 인덱스와 해당 토큰 사이의 맵을 만든 후 다음 마스킹할 단어를 무작위로 결정하고 입력에 해당 마스크를 적용한다. 레이블은 마스크 단어에 해당하는 레이블을 제외하고 모두 -100이다.

In [30]:
import collections
import numpy as np

from transformers import default_data_collator

wwm_probability = 0.2


def whole_word_masking_data_collator(features):
	for feature in features:
		word_ids = feature.pop("word_ids") 
		
		# 단어와 해당 토큰 인덱스 간의 map 생성
		mapping = collections.defaultdict(list)
		current_word_index = -1
		current_word = None
		
		for idx, word_id in enumerate(word_ids):
			if word_id is not None:
				if word_id != current_word:
					# 새 단어가 시작된 경우
					current_word = word_id
					current_word_index += 1
				mapping[current_word_index].append(idx)
		
		r"""
        feature의 word_id가 [0, 0, 1, 2, 3, 4, 5]인 경우 mapping의 형태
        mapping = {
        	단어 인덱스: [토큰 인덱스들]
            "0": [0, 1],
            "1": [2],
            "2": [3],
            ...
            "5": [6]
        }
        word_id와 mapping되는 token_id를 mapping
        """
		
		# =================
		# 무작위로 단어 마스킹
		# =================
		
		# 이산 확률 분포 (= 이항 분포)
		# n = 1이 나올 확률이 0.2인 동전 던지기를 len(mapping)만큼 수행
		mask = np.random.binomial(1, wwm_probability, (len(mapping),))
		
		input_ids = feature["input_ids"]
		labels = feature["labels"]
		new_labels = [-100] * len(labels)
		
		for word_id in np.where(mask)[0]:
			word_id = word_id.item()  # [0, 0, 0, 0, 1, ...]의 index
			for idx in mapping[word_id]:
				new_labels[idx] = labels[idx]  # 정답인 label은 따로 저장
				input_ids[idx] = tokenizer.mask_token_id  # 마스킹
		
                	
	return default_data_collator(features)


# test
samples = [lm_datasets["train"][i] for i in range(2)]
batch = whole_word_masking_data_collator(samples)

for chunk in batch["input_ids"]:
	print(f">>> {tokenizer.decode(chunk)}")

>>> [CLS] i [MASK] i [MASK] curious [MASK] yellow from my video [MASK] because of [MASK] the [MASK] [MASK] surrounded it [MASK] [MASK] was first released in 1967. i also heard that [MASK] [MASK] [MASK] [MASK] seized by u. s. [MASK] [MASK] it ever tried to enter this country, [MASK] [MASK] a [MASK] of [MASK] considered " controversial " i really had to see [MASK] for myself. < br / > < br / [MASK] the plot is centered around a young swedish [MASK] [MASK] [MASK] [MASK] who wants to learn everything she can about life [MASK] in particular [MASK] [MASK] to focus her attentions to making some sort of documentary on [MASK] the average swede thought about certain [MASK] issues [MASK]
>>> [MASK] the [MASK] war and race [MASK] in the [MASK] states [MASK] [MASK] [MASK] [MASK] politicians [MASK] ordinary denizens of [MASK] [MASK] their [MASK] on politics, she has sex with her [MASK] teacher [MASK] classmates, and married men. < [MASK] / [MASK] < br [MASK] > what [MASK] me about i [MASK] curious -

이제 두 개의 data collator가 있으니 나머지 fine-tuning 단계는 동일하다. 먼저 training dataset을 수천개의 예제만 포함하도록 다운샘플링한다.

In [31]:
train_size = 10_000
test_size = int(train_size / 10)

downsampled_dataset = lm_datasets["train"].train_test_split(
		train_size = train_size, test_size = test_size, seed = 42
)
downsampled_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 10000
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 1000
    })
})

### TrainingArguments

In [39]:
from transformers import TrainingArguments

batch_size = 64

# Show the training loss with every epoch
logging_steps = len(downsampled_dataset["train"]) // batch_size
model_name = model_checkpoint.split("/")[-1]

training_dc_args = TrainingArguments(
		output_dir = f"../../models/7.2/{model_name}-finetuned-imdb",
		overwrite_output_dir = True,
		evaluation_strategy = "epoch",
		learning_rate = 2e-5,
		weight_decay = 0.01,
		per_device_train_batch_size = batch_size,
		per_device_eval_batch_size = batch_size,
		fp16 = True,
		logging_steps = logging_steps,
		optim = "adamw_torch", # warning 방지용
)

training_wwmdc_args = TrainingArguments(
		output_dir = f"../../models/7.2/{model_name}-finetuned-imdb",
		overwrite_output_dir = True,
		evaluation_strategy = "epoch",
		learning_rate = 2e-5,
		weight_decay = 0.01,
		per_device_train_batch_size = batch_size,
		per_device_eval_batch_size = batch_size,
		fp16 = True,
		logging_steps = logging_steps,
		optim = "adamw_torch", # warning 방지용
		remove_unused_columns = False # 이 새끼가 자꾸 word_ids 칼럼 지워서 에러났음 default True
)

### Trainer

In [40]:
from transformers import Trainer

trainer_data_collator = Trainer(
		model = model,
		args = training_dc_args,
		train_dataset = downsampled_dataset["train"],
		eval_dataset = downsampled_dataset["test"],
		data_collator = data_collator
)

trainer_wwmdc = Trainer(
		model = model,
		args = training_wwmdc_args,
		train_dataset = downsampled_dataset["train"],
		eval_dataset = downsampled_dataset["test"],
		data_collator = whole_word_masking_data_collator,
)


## 언어 모델을 위한 Preplexity

학습할 레이블이 표기된 말뭉치가 제공되는 text classification 또는 question answering같은 작업과 달리, 언어 모델링에는 명시적 레이블이 없다.

좋은 언어 모델은 문법적으로 정확한 문장에 높은 확률을 할당하고 말도 안되는 문장에 낮은 확률을 할당한다. 이 말도 안되는 문장이 어떻게 생겼는지에 대한 더 나은 아이디어를 제공하기 위해 온라인에서 **자동 고침 실패(autocorrect fails)** 전체 데이터셋을 찾을 수 있다. 이 데이터셋은 사람의 전화에 있는 모델이 다소 웃기고 종종 부적절한 문장을 생성한 예시들을 볼 수 있다.

테스트셋이 대부분 문법적으로 올바른 문장으로 구성되어 있다고 가정하면 언어 모델의 품질을 측정하는 한 가지 방법은 테스트셋의 모든 문장에서 다음 단어에 할당할 확률을 계산하는 것이다. 높은 확률은 모델이 새로운 예시에 놀라거나(surprised) 당황하지(preplexed) 않았음을 나타내며 언어의 기본 문법 패턴을 학습했음을 나타낸다. 이 preplexity에 대한 다양한 수학적 정의가 있지만 이번에 사용할 것은 cross-entropy loss의 exponential로 정의한다.

따라서, `Trainer.evaluate()`함수를 사용해 테스트셋에 대한 cross-entropy loss를 계산한 다음 exponential을 취해 pre-trained model의 복잡도를 계산할 수 있다.

In [41]:
import math

r"""
Preplexity는 모델이 예측하지 못해 당황스럽다는 뜻이므로 작을수록 좋다.
training 후 preplexity가 낮아지는지 확인해본다.
"""

# 1. data_collator 사용
eval_results = trainer_data_collator.evaluate()
print(f">>> Perplexity before data collator evaluate: {math.exp(eval_results['eval_loss']):.2f}")
trainer_data_collator.train()
eval_results = trainer_data_collator.evaluate()
print(f">>> Perplexity after data collator evaluate: {math.exp(eval_results['eval_loss']):.2f}")


# 2. whole_word_masking_data_collator 사용
eval_results = trainer_wwmdc.evaluate()
print(f">>> Perplexity before whole word masking data collator evaluate: {math.exp(eval_results['eval_loss']):.2f}")
trainer_wwmdc.train()
eval_results = trainer_wwmdc.evaluate()
print(f">>> Perplexity after whole word masking data collator evaluate: {math.exp(eval_results['eval_loss']):.2f}")

>>> Perplexity before data collator evaluate: 18.78


Epoch,Training Loss,Validation Loss


>>> Perplexity after data collator evaluate: 10.85


>>> Perplexity before whole word masking data collator evaluate: 2.08


Epoch,Training Loss,Validation Loss


>>> Perplexity after whole word masking data collator evaluate: 1.90


## &#129303;Accelerate를 활용한 DistilBERT Fine-Tuning

위에서 무작위 마스킹이 적용된다는 것을 확인했다. 따라서 각 학습 실행에서 perplexity 점수에 변동이 있을 것이다. 이 무작위성의 원인을 제거하는 한 가지 방법은 전체 테스트 집합에 대해서 마스킹을 한 번만 적용한 다음 &#129303;Transformers의 기본 data collator를 사용해 평가하는 것이다. 이를 어떻게 적용하는지 보기 위해 `DataCollatorForLanguageModeling`을 처음 접했을 때와 유사하게 개별 batch에 마스킹을 적용하는 간단한 함수를 구현해본다.

In [50]:
def insert_random_mask(batch):
	features = [dict(zip(batch, t)) for t in zip(*batch.values())]
	masked_inputs = data_collator(features)
	
	# 데이터셋의 각 칼럼에 대해서 새로운 "masked" 칼럼을 생성
	return {f"masked_{k}" : v.numpy() for k, v in masked_inputs.items()}

이 함수를 테스트 집합에 적용하고 마스크되지 않은 열을 삭제해 마스크된 열로 교체한다.

In [59]:
if "word_ids" in downsampled_dataset.keys():
	downsampled_dataset = downsampled_dataset.remove_columns(["word_ids"])

eval_dataset = downsampled_dataset["test"].map(
		insert_random_mask,
		batched = True,
		remove_columns = downsampled_dataset["test"].column_names
)
eval_dataset = eval_dataset.rename_columns(
		{
				"masked_input_ids": "input_ids",
				"masked_attention_mask": "attention_mask",
				"masked_labels": "labels"
		}
)

이전과 같이 DataLoader를 설정할 수 있지만 평가 집합에 대해 &#129303;Transformers의 `default_data_collator`를 사용한다.

In [60]:
from torch.utils.data import DataLoader

batch_size = 64
train_data_loader = DataLoader(
		downsampled_dataset["train"],
		shuffle = True,
		batch_size = batch_size,
		collate_fn = data_collator
)
eval_data_loader = DataLoader(
		eval_dataset, batch_size = batch_size, collate_fn = default_data_collator
)

In [61]:
# Load Pre-Trained Model
model = AutoModelForMaskedLM.from_pretrained(model_checkpoint)

# AdamW Optimizer
from torch.optim import AdamW

optimizer = AdamW(model.parameters(), lr = 5e-5)

# Accelerate
from accelerate import Accelerator

accelerator = Accelerator()
model, optimizer, train_data_loader, eval_data_loader = accelerator.prepare(
		model, optimizer, train_data_loader, eval_data_loader
)

# Sheduler
from transformers import get_scheduler

num_train_epochs = 3
num_update_steps_per_epoch = len(train_data_loader)
num_training_steps = num_train_epochs * num_update_steps_per_epoch

lr_scheduler = get_scheduler(
		"linear",
		optimizer = optimizer,
		num_warmup_steps = 0,
		num_training_steps = num_training_steps
)

### Training Loop

In [63]:
from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps))

for epoch in range(num_train_epochs):
	model.train()
	for batch in train_data_loader:
		outputs = model(**batch)
		loss = outputs.loss
		accelerator.backward(loss)
		
		optimizer.step()
		lr_scheduler.step()
		optimizer.zero_grad()
		progress_bar.update(1)
	
	model.eval()
	losses = []
	for batch in eval_data_loader:
		with torch.no_grad():
			outputs = model(**batch)
		
		loss = outputs.loss
		losses.append(accelerator.gather(loss.repeat(batch_size)))
		
	losses = torch.cat(losses) # Concatenate
	losses = losses[: len(eval_dataset)]
	
	try:
		perplexity = torch.exp(torch.mean(losses))
	except OverflowError:
		perplexity = float("inf")
	
	print(f">>> Epoch {epoch}: Perplexity = {perplexity}")
	
	# Save Checkpoint
	accelerator.wait_for_everyone()
	unwrapped_model = accelerator.unwrap_model(model)
	unwrapped_model.save_pretrained("../../models/7.2/model_accelerated", save_function = accelerator.save)
	if accelerator.is_main_process:
		tokenizer.save_pretrained("../../models/7.2/model_accelerated")

  0%|          | 0/471 [00:00<?, ?it/s]

>>> Epoch 0: Perplexity = 11.307821273803711
>>> Epoch 1: Perplexity = 11.092926979064941
>>> Epoch 2: Perplexity = 11.092926979064941


## Use Fine-Tuned Model

In [64]:
from transformers import pipeline

mask_filler = pipeline(
		"fill-mask",
		model = "../../models/7.2/model_accelerated"
)

text = "This is a great [MASK]"
preds = mask_filler(text)

for pred in preds:
	print(f">>> {pred['sequence']}")

>>> this is a great!
>>> this is a great.
>>> this is a great film
>>> this is a great movie
>>> this is a great adventure
